In [9]:
import pandas as pd


# Index for the DataFrame
index = [ "gpt-4-0613", "mixtral-8x7b-instruct-v0.1", "gpt-3.5-turbo-0613",
          "text-bison-001 (PaLM-2 text)", "llama-2-70B-chat", "pmc-llama-13b"]

data = {
    "No bias":          [0.727, 0.518, 0.497, 0.429, 0.357, 0.334],
    "Self-diagnosis":   [0.698, 0.460, 0.288, 0.322, 0.169, 0.197],
    "Recency":          [0.679, 0.455, 0.333, 0.232, 0.141, 0.247],
    "Confirmation":     [0.725, 0.497, 0.407, 0.318, 0.177, 0.250],
    "Frequency":        [0.627, 0.373, 0.274, 0.167, 0.104, 0.224],
    "Cultural":         [0.681, 0.421, 0.277, 0.231, 0.207, 0.278],
    "Status quo":       [0.679, 0.445, 0.361, 0.220, 0.160, 0.290],
    "False consensus":  [0.000, 0.428, 0.239, 0.213, 0.139, 0.155]
}

# Creating the DataFrame
no_mitigation_df = pd.DataFrame(data, index=index)
print(no_mitigation_df)

data = {
    "No bias":          [0.500, 0.000, 0.000, 0.000, 0.000, 0.000],
    "Self-diagnosis":   [0.500, 0.460, 0.288, 0.322, 0.169, 0.197],
    "Recency":          [0.500, 0.455, 0.333, 0.232, 0.141, 0.247],
    "Confirmation":     [0.500, 0.497, 0.407, 0.318, 0.177, 0.250],
    "Frequency":        [0.500, 0.373, 0.274, 0.167, 0.104, 0.224],
    "Cultural":         [0.500, 0.421, 0.277, 0.231, 0.207, 0.278],
    "Status quo":       [0.500, 0.445, 0.361, 0.220, 0.160, 0.290],
    "False consensus":  [0.500, 0.428, 0.239, 0.213, 0.139, 0.155]
}

mitigation_df = pd.DataFrame(data, index=index)
print(mitigation_df)

                              No bias  Self-diagnosis  Recency  Confirmation  \
gpt-4-0613                      0.727           0.698    0.679         0.725   
mixtral-8x7b-instruct-v0.1      0.518           0.460    0.455         0.497   
gpt-3.5-turbo-0613              0.497           0.288    0.333         0.407   
text-bison-001 (PaLM-2 text)    0.429           0.322    0.232         0.318   
llama-2-70B-chat                0.357           0.169    0.141         0.177   
pmc-llama-13b                   0.334           0.197    0.247         0.250   

                              Frequency  Cultural  Status quo  False consensus  
gpt-4-0613                        0.627     0.681       0.679            0.000  
mixtral-8x7b-instruct-v0.1        0.373     0.421       0.445            0.428  
gpt-3.5-turbo-0613                0.274     0.277       0.361            0.239  
text-bison-001 (PaLM-2 text)      0.167     0.231       0.220            0.213  
llama-2-70B-chat                  

In [4]:
def query_results_df(results_df, model):
    curr_model = results_df.loc[model]

    cols = results_df.columns

    r = []
    theta = []
    for col in cols:
        if curr_model[col] != "":
            r.append(curr_model[col])
            theta.append(col)
    df = pd.DataFrame(dict(r=r, theta=theta))
    return df

In [42]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Define a base color for the plots
base_color = 'blue'

# Determine the number of plots/models
num_models = len(index)

# Specify the number of rows as an input variable
num_rows = 2  # Change this to your desired number of rows

# Calculate the number of columns needed based on the number of models and rows
num_cols = -(-num_models // num_rows)  # Ceiling division to ensure enough columns

# Create a subplot with specified number of rows and columns
fig = make_subplots(rows=num_rows, cols=num_cols,
                    specs=[[{'type': 'polar'}] * num_cols] * num_rows,
                    vertical_spacing=0.35)  # Increase the vertical spacing to position titles better

# Loop through each model and add both non-mitigated and mitigated data to the same subplot
for i, model in enumerate(index):
    # Determine the row and column to place each subplot
    row = (i // num_cols) + 1
    col = (i % num_cols) + 1

    # Non-Mitigated Data
    df_no_mitigation = query_results_df(no_mitigation_df, model)
    fig.add_trace(go.Scatterpolar(
        r=df_no_mitigation['r'],
        theta=df_no_mitigation['theta'],
        fill='toself',
        marker_color=px.colors.sequential.Blues[8],  # Lighter shade of blue
        mode='lines+markers',  # Display lines and markers
    ), row=row, col=col)

    # Mitigated Data
    df_mitigation = query_results_df(mitigation_df, model)
    fig.add_trace(go.Scatterpolar(
        r=df_mitigation['r'],
        theta=df_mitigation['theta'],
        fill='toself',
        marker_color=px.colors.sequential.Blues[8],  # Darker shade of blue
        mode='lines+markers',  # Display lines and markers
        opacity=0.7  # Slightly transparent for distinction
    ), row=row, col=col)

# Hide the legend
fig.update_layout(showlegend=False)

# Customize layout with increased height to accommodate the vertical spacing
fig.update_layout(
    height=1000,  # Increase the height to provide enough space for titles
    margin=dict(t=20, b=20)  # Adjust top and bottom margins as necessary
)

# Add custom annotations for subplot titles to better control their position
annotations = []
for i in range(num_models):
    # Calculate the position of each subplot title
    x_pos = ((i % num_cols) + 0.5) / num_cols
    y_pos = 1 - ((i // num_cols) * vertical_spacing) - 0.03  # Adjust y_pos for titles

    annotations.append(dict(
        text=f"Model {i+1}",  # Subplot Title
        x=x_pos,  # X position (center of subplot)
        y=y_pos,  # Y position (above subplot)
        xref="paper",
        yref="paper",
        showarrow=False,
        font=dict(size=12),  # Increase font size
        align='center'
    ))

fig.update_layout(annotations=annotations)

fig.show()


In [7]:
import plotly.graph_objects as go
import plotly.express as px

# Define a set of distinct colors for the plots
colors = px.colors.qualitative.Set1

# Create a single polar plot with a specified width
fig = go.Figure(layout=go.Layout(width=700))  # Adjust width as needed

for i, model in enumerate(index):
    df = query_results_df(no_mitigation_df, model)
    # Add each model's data to the plot with a unique color and label for the legend
    fig.add_trace(go.Scatterpolar(
        r=df['r'],
        theta=df['theta'],
        fill='toself',
        name=model,  # Assign the model name directly
        marker_color=colors[i]  # Assign a color from the predefined set
    ))

# Update layout for a more compact view and clear legend display
fig.update_layout(
    title="Model Comparison",
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 1]
        )
    ),
    legend_title="Models",
    legend=dict(
        orientation="h",  # Horizontal orientation
        yanchor="bottom",  # Anchor point is at the bottom
        y=-0.25,  # Position the legend below the plot
        xanchor="left",  # Anchor point is on the left
        x=0  # Position the legend at the far left
    )
)

fig.show()
